In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///ecommerce.db', echo=True)

In [5]:
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))
    # Fetch and print the result
    print(result.scalar()) # This should print 1 if the connection is successful

2025-08-05 11:55:18,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 11:55:18,108 INFO sqlalchemy.engine.Engine SELECT 1
2025-08-05 11:55:18,109 INFO sqlalchemy.engine.Engine [generated in 0.00316s] ()
1
2025-08-05 11:55:18,111 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()

users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('email', String, unique=True))

products = Table('products', metadata,
                 Column('id', Integer, primary_key=True),
                 Column('name', String),
                 Column('price', Integer))

orders = Table('orders', metadata,
               Column('id', Integer, primary_key=True),
               Column('user_id', Integer, ForeignKey('users.id')))

order_items = Table('order_items', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('order_id', Integer, ForeignKey('orders.id')),
                    Column('product_id', Integer, ForeignKey('products.id')),
                    Column('quantity', Integer))

# Create all tables
metadata.create_all(engine)

2025-08-05 13:14:05,121 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 13:14:05,127 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-08-05 13:14:05,129 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-05 13:14:05,133 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-08-05 13:14:05,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-05 13:14:05,135 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-08-05 13:14:05,136 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-05 13:14:05,139 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-08-05 13:14:05,140 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-05 13:14:05,141 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-08-05 13:14:05,142 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-05 13:14:05,144 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-08-05 13:14:05,145 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [7]:
def add_user(name, email):
    ins = users.insert().values(name=name, email=email)
    with engine.connect() as conn:
        result = conn.execute(ins)
        conn.commit()
        return result.inserted_primary_key[0]  # returns the new user's id

In [8]:
add_user('John Doe', 'john.doe@example.com')

2025-08-05 13:18:14,887 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 13:18:14,889 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-08-05 13:18:14,890 INFO sqlalchemy.engine.Engine [generated in 0.00330s] ('John Doe', 'john.doe@example.com')
2025-08-05 13:18:14,896 INFO sqlalchemy.engine.Engine COMMIT


1

In [11]:
def get_user(id):
    sel = users.select().where(users.c.id == id)
    with engine.connect() as conn:
        result = conn.execute(sel).fetchone()
        return dict(result._mapping) if result else None

In [12]:
get_user(1)  # Assuming the user with ID 1 exists

2025-08-05 13:22:56,829 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 13:22:56,831 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2025-08-05 13:22:56,832 INFO sqlalchemy.engine.Engine [cached since 126.4s ago] (1,)
2025-08-05 13:22:56,833 INFO sqlalchemy.engine.Engine ROLLBACK


{'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}

In [14]:
def update_user(user_id, name=None, email=None):
    update_values = {}
    if name is not None:
        update_values['name'] = name
    if email is not None:
        update_values['email'] = email
    if not update_values:
        return False  # Nothing to update

    upd = users.update().where(users.c.id == user_id).values(**update_values)
    with engine.connect() as conn:
        result = conn.execute(upd)
        conn.commit()
        return result.rowcount > 0  # True if a row was updated, else False

In [15]:
update_user(1, name='Jane Doe', email='john.new@example.com')

2025-08-05 13:26:42,753 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 13:26:42,754 INFO sqlalchemy.engine.Engine UPDATE users SET name=?, email=? WHERE users.id = ?
2025-08-05 13:26:42,755 INFO sqlalchemy.engine.Engine [generated in 0.00250s] ('Jane Doe', 'john.new@example.com', 1)
2025-08-05 13:26:42,759 INFO sqlalchemy.engine.Engine COMMIT


True

In [16]:
def delete_user(user_id):
    delete_stmt = users.delete().where(users.c.id == user_id)
    with engine.connect() as conn:
        result = conn.execute(delete_stmt)
        conn.commit()
        return result.rowcount > 0  # True if a row was deleted, else False

In [17]:
delete_user(1)  # Assuming the user with ID 1 exists

2025-08-05 13:29:00,389 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 13:29:00,391 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2025-08-05 13:29:00,393 INFO sqlalchemy.engine.Engine [generated in 0.00365s] (1,)
2025-08-05 13:29:00,397 INFO sqlalchemy.engine.Engine COMMIT


True